In [ ]:
!pip install keras.datas

In [1]:
import time
import numpy as np
import os
import pickle
import matplotlib.pyplot as plt
from keras.datasets import mnist

ModuleNotFoundError: No module named 'keras'

In [ ]:
class Layer_Dense:
    def __init__(self, n_inputs, n_neurons):
        self.weights = 0.10 * np.random.randn(n_inputs, n_neurons)
        self.biases = np.zeros((1, n_neurons))

    def forward(self, inputs):
        self.inputs = inputs
        self.output = np.dot(inputs, self.weights) + self.biases

    def backward(self, dvalues):
        self.dweights = np.dot(self.inputs.T, dvalues)
        self.dbiases = np.sum(dvalues, axis=0, keepdims=True)
        self.dinputs = np.dot(dvalues, self.weights.T)



In [ ]:
class ActivationReLU:
    def forward(self, inputs):
        self.inputs = inputs 
        self.output = np.maximum(0, inputs)

    def backward(self, dvalues):
        self.dinputs = dvalues.copy()
        self.dinputs[self.inputs <= 0] = 0

In [ ]:
class ActivationSoftMax:
    def forward(self, inputs):
        self.exp_values = np.exp(inputs - np.max(inputs, axis=1, keepdims=True))
        self.probabilities = self.exp_values / np.sum(self.exp_values, axis=1, keepdims=True)
        self.output = self.probabilities

    def backward(self, dvalues):
        self.dinputs = np.empty_like(dvalues)
        for index, (single_output, single_dvalues) in enumerate(zip(self.output, dvalues)):
            single_output = single_output.reshape(-1, 1)
            jacobian_matrix = np.diagflat(single_output) - np.dot(single_output, single_output.T)
            self.dinputs[index] = np.dot(jacobian_matrix, single_dvalues)

In [ ]:
class Loss:
    def calculate(self, output, y):
        self.sample_losses = self.forward(output, y)
        self.data_loss = np.mean(self.sample_losses)
        return self.data_loss

In [ ]:
class LossCategoricalCrossEntropy(Loss):
    def forward(self, y_pred, y_true):
        samples = len(y_pred)
        y_pred_clipped = np.clip(y_pred, 1e-7, 1 - 1e-7)

        if y_true.ndim == 1:
            correct_confidences = y_pred_clipped[range(samples), y_true]
        elif y_true.ndim == 2:
            correct_confidences = np.sum(y_pred_clipped * y_true, axis=1)

        negative_log_likelihoods = -np.log(correct_confidences)
        return negative_log_likelihoods

    def backward(self, dvalues, y_true):
        samples = len(dvalues)
        labels = len(dvalues[0])

        if y_true.ndim == 1:
            y_true = np.eye(labels)[y_true]
        
        if np.any(dvalues == 0):
            # Handle zero division cases (e.g., set dinputs to a small constant)
            self.dinputs = np.where(dvalues == 0, 1e-8, -y_true / dvalues)
        else:
            self.dinputs = -y_true / dvalues
        self.dinputs = self.dinputs / samples

In [ ]:
class Optimizer_SGD:
    def __init__(self, learning_rate=1.0, decay=0.0, momentum=0.0):
        self.learning_rate = learning_rate
        self.current_learning_rate = learning_rate
        self.decay = decay
        self.iterations = 0
        self.momentum = momentum

    def pre_update_params(self):
        if self.decay:
            self.current_learning_rate = self.learning_rate * (1.0 / (1.0 + self.decay * self.iterations))

    def update_params(self, layer):
        if self.momentum:
            if not hasattr(layer, 'weight_momentums'):
                layer.weight_momentums = np.zeros_like(layer.weights)
                layer.bias_momentums = np.zeros_like(layer.biases)

            weight_updates = self.momentum * layer.weight_momentums - self.current_learning_rate * layer.dweights
            layer.weight_momentums = weight_updates
            bias_updates = self.momentum * layer.bias_momentums - self.current_learning_rate * layer.dbiases
            layer.bias_momentums = bias_updates
        else:
            weight_updates = -self.current_learning_rate * layer.dweights
            bias_updates = -self.current_learning_rate * layer.dbiases

        layer.weights += weight_updates
        layer.biases += bias_updates

    def post_update_params(self):
        self.iterations += 1


In [ ]:
(X_train, Y_train), (X_test, Y_test) = mnist.load_data()
X_train = X_train / 255
X = X_train.reshape(X_train.shape[0], -1)
y = Y_train
# y = np.zeros((Y_train.size, Y_train.max() + 1))
# y[np.arange(Y_train.size), Y_train] = 1



input_layer = Layer_Dense(784, 784)
activation_input = ActivationReLU()

hidden_layer1 = Layer_Dense(784, 128)
activation1 = ActivationReLU()

hidden_layer2 = Layer_Dense(128, 64)
activation2 = ActivationReLU()

hidden_layer3 = Layer_Dense(64, 64)
activation3 = ActivationReLU()

output_layer = Layer_Dense(64, 10)
activation_output = ActivationSoftMax()

loss_function = LossCategoricalCrossEntropy()
optimizer = Optimizer_SGD()

losses = []
accuracies = []
epochs = 10001


In [ ]:
for epoch in range(epochs):

    input_layer.forward(X)
    activation_input.forward(input_layer.output)

    hidden_layer1.forward(activation_input.output)
    activation1.forward(hidden_layer1.output)

    hidden_layer2.forward(activation1.output)
    activation2.forward(hidden_layer2.output)

    hidden_layer3.forward(activation2.output)
    activation3.forward(hidden_layer3.output)

    output_layer.forward(activation3.output)
    activation_output.forward(output_layer.output)

    loss = loss_function.calculate(output_layer.output, y)
    losses.append(loss)

    predictions = np.argmax(activation_output.output, axis=1)
    # print(predictions)
    accuracy = np.mean(predictions == y)
    accuracies.append(accuracy)
    
    loss_function.backward(activation_output.output, y)
    activation_output.backward(loss_function.dinputs)
    output_layer.backward(activation_output.dinputs)
    
    activation3.backward(output_layer.dinputs)
    hidden_layer3.backward(activation3.dinputs)

    activation2.backward(hidden_layer3.dinputs)
    hidden_layer2.backward(activation2.dinputs)
    
    activation1.backward(hidden_layer2.dinputs)
    hidden_layer1.backward(activation1.dinputs)

    activation_input.backward(hidden_layer1.dinputs)
    input_layer.backward(activation_input.dinputs)

    optimizer.pre_update_params()
    optimizer.update_params(input_layer)
    optimizer.update_params(hidden_layer1)
    optimizer.update_params(hidden_layer2)
    optimizer.update_params(hidden_layer3)
    optimizer.update_params(output_layer)
    optimizer.post_update_params()

    if epoch % 100 == 0:
        print(f"Epoch: {epoch}, Loss: {loss:.3f} Accuracy: {accuracy:.3f}")
    


In [ ]:
model_parameters = {
    "input_layer_weights": input_layer.weights,
    "input_layer_biases": input_layer.biases,
    "hidden_layer1_weights": hidden_layer1.weights,
    "hidden_layer1_biases": hidden_layer1.biases,
    "hidden_layer2_weights": hidden_layer2.weights,
    "hidden_layer2_biases": hidden_layer2.biases,
    "hidden_layer3_weights": hidden_layer3.weights,
    "hidden_layer3_biases": hidden_layer3.biases,
    "output_layer_weights": output_layer.weights,
    "output_layer_biases": output_layer.biases,
}

with open("OCR_Model_784,128,64,64,10.pkl", "wb") as f:
    pickle.dump(model_parameters, f)

print("Model saved sucessfully.")



In [ ]:
plt.figure(figsize=(12, 5))
plt.plot(range(epochs), losses, lable="Loss")
plt.xlabel("Eposhes")
plt.ylabel("Loss")
plt.title("Loss over Epoches")
plt.legend()

plt.subplot(1, 2, 3)
plt.plot(range(epochs), accuracies, lable="Accuracy", color="orange")
plt.xlabel("Eposhes")
plt.ylabel("Accuracy")
plt.title("Accuracy over Epoches")

plt.show()
